In [19]:
# make sure to run nltk.download() if you're getting errors
import string
from keras import preprocessing
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split

import numpy as np
import collections
from nltk.tokenize import word_tokenize

So far, this code just takes the data set and creates an embedding layer with randomly initialized word vectors for the positive examples.

# TODO

* The rest of the CNN-rand model
* Encapsulate the model code into a Python class
* K-fold cross-validation pipeline
* Tune hyperparameters to achieve better results?

In [20]:
pos_examples = open('data/rt-polarity.pos', encoding='ISO-8859-1').readlines()
neg_examples = open('data/rt-polarity.neg', encoding='ISO-8859-1').readlines()

vocab = collections.defaultdict(int)
translator = str.maketrans('', '', string.punctuation)

# 1. Tokenize all strings
token_pos = list(map(lambda ex: word_tokenize(ex.translate(translator)), pos_examples))
token_neg = list(map(lambda ex: word_tokenize(ex.translate(translator)), neg_examples))

# 2. Get vocabulary size and max sentence length
max_sentence_length = 0
for ex_p, ex_n in zip(token_pos, token_neg):
    max_sentence_length = max(max_sentence_length, len(ex_p), len(ex_n))
    for word in ex_p:
        vocab[word] += 1
    for word in ex_n:
        vocab[word] += 1
        
vocab_size = len(vocab)
print("Vocabulary size: " + str(vocab_size))
print("Max sentence length: " + str(max_sentence_length))

# 3. One-Hot encode and pad.
encoded_pos = [preprocessing.text.one_hot(ex, vocab_size) for ex in pos_examples]
padded_pos = preprocessing.sequence.pad_sequences(encoded_pos, maxlen=max_sentence_length, padding='post')
encoded_neg = [preprocessing.text.one_hot(ex, vocab_size) for ex in neg_examples]
padded_neg = preprocessing.sequence.pad_sequences(encoded_neg, maxlen=max_sentence_length, padding='post')

print(padded_pos.shape)
print(padded_neg.shape)

X = np.concatenate((padded_pos, padded_neg))
y = np.concatenate((np.ones(padded_pos.shape[0]), np.zeros(padded_neg.shape[0])))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)

# # 4. Create the CNN.
# model = Sequential()
# model.add(Embedding(vocab_size, 128, input_length=max_sentence_length, embeddings_initializer='random_uniform'))


Vocabulary size: 20490
Max sentence length: 51
(5331, 51)
(5331, 51)
(8529, 51) (8529,)
